In [1]:
import os,argparse,time
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.utils.data.distributed

In [2]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [6]:
from dataloaders.deep_moji import DeepMojiDataset
from networks.deepmoji_sa import DeepMojiModel

In [7]:
from networks.eval_metrices import group_evaluation

In [8]:
from tqdm import tqdm, trange

In [9]:
# DataLoader Parameters
params = {'batch_size': 1024,
          'shuffle': True,
          'num_workers': 0}
# Device
device = torch.device("cuda")


In [10]:
class Args:
    use_fp16 = False
    cuda = "cuda"
    hidden_size = 300
    emb_size = 2304
    num_classes = 2
    n_hidden = 2
    adv = False
    adv_level = -1
    lr = 3e-5
    LAMBDA = 1e-4 # adversarial 
    adv_units = 256
    ratio = 0.8
    dropout = 0.5

data_path = #Your data path
args = Args()
args.device = device

In [11]:
split = "train"
train_data = DeepMojiDataset(args, data_path, split, ratio=args.ratio)

Loading preprocessed deepMoji Encoded data
Done, loaded data shapes: (99998, 2304), (99998,), (99998,)


In [12]:
dev_data = DeepMojiDataset(args, data_path, "dev", n=200000, ratio=args.ratio)

Loading preprocessed deepMoji Encoded data
Done, loaded data shapes: (8000, 2304), (8000,), (8000,)


In [13]:
test_data = DeepMojiDataset(args, data_path, "test", n=200000, ratio=args.ratio)

Loading preprocessed deepMoji Encoded data
Done, loaded data shapes: (7998, 2304), (7998,), (7998,)


In [14]:
training_generator = torch.utils.data.DataLoader(train_data, **params)
validation_generator = torch.utils.data.DataLoader(dev_data, **params)
test_generator = torch.utils.data.DataLoader(test_data, **params)

In [15]:
model = DeepMojiModel(args)
model = model.to(device)

In [16]:

from torch.optim import Adam

LEARNING_RATE = args.lr

optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)

In [17]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

scheduler = ReduceLROnPlateau(optimizer, mode = 'max', factor = 0.5, patience = 2)

In [18]:
criterion = torch.nn.CrossEntropyLoss()

In [19]:
def train_epoch(model, iterator, optimizer, criterion, clipping_value, device, args):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in tqdm(iterator):
        
        text = batch[0]
        tags = batch[1].long()
        # tags = batch[2].long() # Reverse
        p_tags = batch[2].float()
        # p_tags = batch[1]

        text = text.to(device)
        tags = tags.to(device)
        p_tags = p_tags.to(device)
        
        optimizer.zero_grad()
        if args.Augmentation:
            predictions = model(text, p_tags)
        else:
            predictions = model(text)
        
        
        loss = criterion(predictions, tags)
                        
        loss.backward()

        torch.nn.utils.clip_grad_norm(model.parameters(), clipping_value)
        
        optimizer.step()
        # print(loss.item())
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [20]:
def train_batch(model, batch, optimizer, criterion, clipping_value, device, args):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    # for batch in tqdm(iterator):
        
    text = batch[0]
    tags = batch[1].long()
    # tags = batch[2].long() # Reverse
    p_tags = batch[2].float()
    # p_tags = batch[1] 
    text = text.to(device)
    tags = tags.to(device)
    p_tags = p_tags.to(device)

    optimizer.zero_grad()

    if args.Augmentation:
        predictions = model(text, p_tags)
    else:
        predictions = model(text)


    loss = criterion(predictions, tags)

    loss.backward() 
    torch.nn.utils.clip_grad_norm(model.parameters(), clipping_value)

    optimizer.step()
    # print(loss.item())
    epoch_loss += loss.item()
        
    return epoch_loss

In [21]:
def eval(model, iterator, criterion, device, args):
    
    epoch_loss = 0
    
    model.eval()
    
    preds = []
    labels = []
    private_labels = []

    for batch in iterator:
        
        text = batch[0]

        tags = batch[1]
        # tags = batch[2] #Reverse
        p_tags = batch[2]

        text = text.to(device)
        tags = tags.to(device).long()
        p_tags = p_tags.to(device).float()
        
        if args.Augmentation:
            predictions = model(text, p_tags)
        else:
            predictions = model(text)
        
        loss = criterion(predictions, tags)
                        
        epoch_loss += loss.item()
        
        predictions = predictions.detach().cpu()
        tags = tags.cpu().numpy()

        preds += list(torch.argmax(predictions, axis=1).numpy())
        labels += list(tags)

        private_labels += list(batch[2].cpu().numpy())
        # private_labels += list(batch[1].cpu().numpy()) # Reverse

    
    return ((epoch_loss / len(iterator)), preds, labels, private_labels)

In [22]:
from sklearn.metrics import accuracy_score


In [23]:
best_loss, preds, labels, p_labels = eval(model, validation_generator, criterion, device, args)
best_acc = accuracy_score(preds, labels)
best_epoch = -1
for i in range(60):
    train_epoch(model, training_generator, optimizer, criterion, 1, device, args)
    valid_loss, preds, labels, p_labels = eval(model, validation_generator, criterion, device, args)
    # learning rate scheduler
    scheduler.step(valid_loss)
    epoch_acc = accuracy_score(preds, labels)
    if valid_loss < best_loss:
        if i >= 2:
            print(i)
            best_loss = valid_loss
            best_epoch = i
            torch.save(model.state_dict(), "models\\deepmoji_model.pt")
    else:
        if best_epoch+5<=i:
            break
model.load_state_dict(torch.load("models\\deepmoji_model.pt"))

100%|██████████| 98/98 [00:01<00:00, 67.39it/s]


<All keys matched successfully>

In [24]:
test_loss, preds, labels, p_labels = eval(model, test_generator, criterion, device, args)
accuracy_score(preds, labels)

0.7154288572143036

In [25]:
preds = np.array(preds)
labels = np.array(labels)
p_labels = np.array(p_labels)

In [26]:
eval_metrices = group_evaluation(preds, labels, p_labels, silence=False)

Accuracy 0: 0.75575
Accuracy 1: 0.675087543771886
TPR 0: 0.601
TPR 1: 0.92096048024012
TNR 0: 0.9105
TNR 1: 0.42921460730365185
TPR gap: -0.31996048024012
TNR gap: 0.4812853926963481


In [27]:
(eval_metrices["Accuracy_0"]+eval_metrices["Accuracy_1"])/2

0.715418771885943

# Leakage Evaluation

In [28]:
from networks.eval_metrices import leakage_evaluation

In [29]:
leakage_evaluation(model, 
                    0, 
                    training_generator,
                    validation_generator,
                    test_generator,
                    device)

Dev Accuracy: 0.689625
Test Accuracy: 0.7005501375343836


In [30]:
leakage_evaluation(model, 
                    -1, 
                    training_generator,
                    validation_generator,
                    test_generator,
                    device)

Dev Accuracy: 0.839875
Test Accuracy: 0.8475868967241811
